In [ ]:
import math
import numpy as np
from sklearn import datasets

def entropy(p1,n1): #算亂度
  if (p1==0 and n1==0):
    return 1
  elif (p1==0 or n1==0):
    return 0
  else:
    pp = p1/(p1+n1)
    np = n1/(p1+n1)
    return -pp*math.log2(pp)-np*math.log2(np)

def IG(p1,n1,p2,n2):  #information gain 分類方法對降低亂度的幫助大小
  num1 = p1+n1
  num2 = p2+n2
  num = num1+num2
  return entropy(p1+p2,n1+n2)-num1/num*entropy(p1,n1)-num2/num*entropy(p2,n2)

def ID3DTtrain(feature,target):
  node = dict()
  node['data'] = range(len(target)) #全部資料放在根節點，紀錄有多少資料落在這裡
  tree = [] #要output的樹
  tree.append(node) #把根結點塞進去

  t = 0 #一個一個決定結點的內容是否要分枝成兩個
  while (t<len(tree)):  #表示t還沒看到最後一個節點
    index = tree[t]['data'] #從0號根結點開始

    if(sum(target[index])==0):  #target裡面都是0、1、2，如果全為0就代表分到一群都是0的
      tree[t]['leaf'] = 1
      tree[t]['decision'] = 0
    elif(sum(target[index])==len(index)):
      tree[t]['leaf'] = 1
      tree[t]['decision'] = 1
    else:
      bestIG = 0
      for i in range(feature.shape[1]):
        pool = list(set(feature[index,i]))  #index=第t個node中所有資料的編號，把第i個特徵拿出來比較
        pool.sort()

        for j in range(len(pool)-1):  #找兩兩的數，中間下去分切(1,3,5、切2,4)，間隔有len(pool)-1個
          thres = (pool[j]+pool[j+1])/2
          G1 = []
          G2 = []
          for k in index:
            if(feature[k][i]<thres):
              G1.append(k)
            else:
              G2.append(k)
          p1 = sum(target[G1]==1)
          p2 = sum(target[G1]==0)
          n1 = sum(target[G2]==1)
          n2 = sum(target[G2]==0)
          thisIG = IG(p1,p2,n1,n2)
          if(thisIG>bestIG):
            bestIG = thisIG
            bestG1 = G1
            bestG2 = G2
            bestthres = thres
            bestf = i
      if(bestIG>0): #還能再切分
        tree[t]['leaf'] = 0
        tree[t]['selectf'] = bestf
        tree[t]['threshold'] = bestthres
        tree[t]['child'] = [len(tree),len(tree)+1]
        node = dict()
        node['data'] = bestG1 #假如現在t=0 切完的右半邊會放在這裡(t=1)
        tree.append(node)
        node = dict()
        node['data'] = bestG2 #左半邊放這裡
        tree.append(node)

      else:
        tree[t]['leaf'] = 1
        if(sum(target[index]==1)>sum(target[index]==0)):  #target=1的數量和=2的數量比較
          tree[t]['decision'] = 1
        else:
          tree[t]['decision'] = 0
    t += 1
  return tree

def ID3DTtest(tree,feature1):
  now = 0
  while(tree[now]['leaf']==0):
    bestf = tree[now]['selectf']
    thres = tree[now]['threshold']
    if(feature1[bestf]<thres):
      now = tree[now]['child'][0]
    else:
      now = tree[now]['child'][1]
  return tree[now]['decision']

data = datasets.load_iris()
feature = data['data']  #每一朵的特色
target = data['target'] #花的種類(有三種各50朵)

#前100朵
T = ID3DTtrain(feature[0:100,:],target[0:100])

T_next = ID3DTtrain(feature[50:150,:],target[50:150]-1)


In [ ]:
IG(p1,n1,p2,n2)

In [ ]:

T_next = ID3DTtrain(feature[50:150,:],target[50:150]-1)
miss = 0  #計算判斷錯誤的次數
for i in range(50,150):
  flag = ID3DTtest(T_next,feature[i,:])
  if(i<100 and flag==0):
    print(i,': ',flag,'correct')
  elif(i<100 and flag==1):
    print(i,': ',flag,'wrong')
    miss+=1

  if(i>=100 and flag==1):
    print(i,': ',flag,'correct')
  elif(i>=100 and flag==0):
    print(i,': ',flag,'wrong')
    miss+=1
print('Accuration',(1-(miss/100))*100,'%')

50 :  0 correct
51 :  0 correct
52 :  0 correct
53 :  0 correct
54 :  0 correct
55 :  0 correct
56 :  0 correct
57 :  0 correct
58 :  0 correct
59 :  0 correct
60 :  0 correct
61 :  0 correct
62 :  0 correct
63 :  0 correct
64 :  0 correct
65 :  0 correct
66 :  0 correct
67 :  0 correct
68 :  0 correct
69 :  0 correct
70 :  0 correct
71 :  0 correct
72 :  0 correct
73 :  0 correct
74 :  0 correct
75 :  0 correct
76 :  0 correct
77 :  0 correct
78 :  0 correct
79 :  0 correct
80 :  0 correct
81 :  0 correct
82 :  0 correct
83 :  0 correct
84 :  0 correct
85 :  0 correct
86 :  0 correct
87 :  0 correct
88 :  0 correct
89 :  0 correct
90 :  0 correct
91 :  0 correct
92 :  0 correct
93 :  0 correct
94 :  0 correct
95 :  0 correct
96 :  0 correct
97 :  0 correct
98 :  0 correct
99 :  0 correct
100 :  1 correct
101 :  1 correct
102 :  1 correct
103 :  1 correct
104 :  1 correct
105 :  1 correct
106 :  1 correct
107 :  1 correct
108 :  1 correct
109 :  1 correct
110 :  1 correct
111 :  1 corr

In [ ]:

x = feature[50:80,:]
y = feature[100:130,:]
p = target[50:80]
q = target[100:130]
feature_new = np.concatenate((x,y),axis=0)  #把兩類的前30筆，合併起來
target_new = np.concatenate((p,q),axis=0)
miss = 0  #計算判斷錯誤的次數
T_type2 = ID3DTtrain(feature_new[0:60,:],target_new[0:60]-1)
for i in range(80,100):
  flag = ID3DTtest(T_type2,feature[i,:])
  if(flag==0):
    print(i,': ',flag,'correct')
  elif(flag==1):
    print(i,': ',flag,'wrong')
    miss+=1
print("-----------------")
for i in range(130,150):
  flag = ID3DTtest(T_type2,feature[i,:])
  if(flag==0):
    print(i,': ',flag,'wrong')
    miss+=1
  elif(flag==1):
    print(i,': ',flag,'correct')
print('Accuration',(1-(miss/40))*100,'%')

80 :  0 correct
81 :  0 correct
82 :  0 correct
83 :  1 wrong
84 :  0 correct
85 :  1 wrong
86 :  0 correct
87 :  0 correct
88 :  0 correct
89 :  0 correct
90 :  0 correct
91 :  0 correct
92 :  0 correct
93 :  0 correct
94 :  0 correct
95 :  0 correct
96 :  0 correct
97 :  0 correct
98 :  0 correct
99 :  0 correct
-----------------
130 :  1 correct
131 :  1 correct
132 :  1 correct
133 :  1 correct
134 :  1 correct
135 :  1 correct
136 :  1 correct
137 :  1 correct
138 :  1 correct
139 :  1 correct
140 :  1 correct
141 :  1 correct
142 :  1 correct
143 :  1 correct
144 :  1 correct
145 :  1 correct
146 :  1 correct
147 :  1 correct
148 :  1 correct
149 :  1 correct
Accuration 95.0 %
